In [1]:
import sys
import numpy as np
import pickle as pkl
import pandas as pd
from operator import itemgetter
sys.setrecursionlimit(2000000)

In [2]:
class VoronoisAlgorithm:
    """
    Runs a modification of Voronoi's Algorithm for Rp-lattices, to find all vertices of Tp.
    Only implemented right now for n = p prime.
    """
    def __init__(self, p=None, dec_approx=1e4):
        self.p = p 
        self.p0 = floor((self.p-2)/2)
        self.dec_approx = dec_approx
        
        # number-theoretic data
        self.K = self._get_cyclo_field()
        self.zeta = self.K.gen()
        self.totally_real_degree = int((self.p-1)/2)
        self.galois_gen = gens(self.K.galois_group())[0]
        self.sigma = [(self.galois_gen)^i for i in range(self.totally_real_degree)]
        self.cc = (self.galois_gen)^(self.totally_real_degree)
        
        # define Xp as a polytope and get the hyperplane equations for its faces
        # useful to store both the hyperplane equation with and without the constant term
        self.Xp = self._get_Xp()
        self.face_matrix_tilde, self.face_matrix = self._get_Xp_face_matrices()
        
        # matrices used to obtain hyperplane equations
        self.M = Matrix(self.K,[[self.sigma[i](((self.zeta)^j+(self.zeta)^(-j))/2) 
                            for i in range(self.totally_real_degree)] 
                            for j in range(self.totally_real_degree)])
        self.M1 = self.M[0,:]
        self.Minv = self.M^(-1)
        
        # to keep track of all of the vertices we find
        self.vertices_of_Tp = set()
        # and all of the inequalities given by elements of Rp
        self.global_inequalities = {}
        
        # for computing orbits of Cn on vectors
        self.Cn_matrix = companion_matrix((self.p)*[1], format='right')
        e1 = vector([1]+(self.p-2)*[0])
        e1_orbit = [e1]
        w = (self.Cn_matrix)*e1
        while w != e1:
            e1_orbit.append(w)
            w = (self.Cn_matrix)*w
        e1_orbit = e1_orbit + [-w for w in e1_orbit]
        self.e1_orbit = e1_orbit

        # data for the Galois action on T_p
        Zp = Integers(p)
        a = Zp.multiplicative_generator()
        self._galois_orbit_idx = [Integer(a*i) for i in range(1,self.p0+1)]

        # placeholder for the polytope T_p
        self.Tp = None
        
    def _get_cyclo_field(self):
        K.<zeta> = CyclotomicField(self.p)
        return K
    
    def _get_Xp(self):
        # defines Xp as a polytope
        vertices = [vector(cos(2*pi*i*j/self.p) 
                    for i in range(1,self.p0+1)) 
                    for j in range(1,self.p0+2)]
        Xp = Polyhedron(vertices, base_ring=AA)
        return Xp
    
    def _get_Xp_face_matrices(self):
        Xp_faces = [f.as_polyhedron() for f in self.Xp.faces(self.p0-1)]
        face_matrix_tilde = Matrix(RR,[f.Hrepresentation()[-1].vector() for f in Xp_faces])
        face_matrix = face_matrix_tilde[:,1:]
        return face_matrix_tilde, face_matrix

    def va_main(self, x0):
        # does Voronoi's algorithm with initial point = x0
        num_vertices = len(self.vertices_of_Tp)
        x0_galois_orbit = {tuple(y) for y in self.galois_orbit(x0)}
        new_orbit_pts = x0_galois_orbit.difference(self.vertices_of_Tp)
        self.vertices_of_Tp = self.vertices_of_Tp.union(new_orbit_pts)
        for x in new_orbit_pts:
            neighbors = {tuple(y) for y in self.neighboring_vertices(vector(x))}
            new_neighbors = neighbors.difference(self.vertices_of_Tp)
            self.vertices_of_Tp = self.vertices_of_Tp.union(new_neighbors)
            for y in new_neighbors:
                self.va_main(x0=vector(y))
        return self.vertices_of_Tp

    def construct_Tp(self):
        # after computing the vertices of T_p using self.va_main, this method
        # constructs T_p as a Polytope
        global_inequalities = self.trim_inequalities()
        Tp = Polyhedron(ieqs=global_inequalities.values(), base_ring=QQ, backend="normaliz")
        self.Tp = Tp
        return Tp

    def trim_inequalities(self):
        gi = {k:v for k,v in self.global_inequalities.items() if not all([a==0 for a in v])}
        gi_new = {}
        for k,v in gi.items():
            if v not in gi_new.values():
                gi_new[k] = v
        self.global_inequalities = gi_new
        return gi_new

    def galois_orbit(self, x):
        Gx = self.pt_to_rnmatrix(x)
        x_tilde = list(vector(Gx[0,:]))
        x_tilde = vector(x_tilde + [-sum(x_tilde)])
        orbit = [x]
        y = vector(itemgetter(*self._galois_orbit_idx)(x_tilde))
        while y != x:
            orbit.append(y)
            Gy = self.pt_to_rnmatrix(y)
            y_tilde = list(vector(Gy[0,:]))
            y_tilde = vector(y_tilde + [-sum(y_tilde)])
            y = vector(itemgetter(*self._galois_orbit_idx)(y_tilde))
        return orbit
    
    def neighboring_vertices(self, x):
        # calculates the neighboring vertices to a vertex x of Tn
        Gx = self.pt_to_rnmatrix(x)
        Gx_int = self.scale_to_integer(Gx)
        _, mvx_orbit_reps = self.min_vecs_cn_orbit_reps(Gx_int)
        mvx_orbit_reps = [rep for rep in mvx_orbit_reps if rep not in self.e1_orbit]
        y1s = self.get_y1s(x, mvx_orbit_reps)
        neighbors = [self.retract_towards_x(y, x, Gx) for y in y1s]
        return neighbors

    def retract_towards_x(self, y, x, Gx):
        # retracts y towards x using PARI/GP quad form methods
        Gy = self.pt_to_rnmatrix(y)
        Gy_int = self.scale_to_integer(Gy)
        Ly_min, v = self.single_min_vec(Gy_int)
        
        while Ly_min < Gy_int[0,0]:
            # just use a single min vec of Ly
            A = v*Gx*v
            scaling_factor = Gy[0,0]/Gy_int[0,0]
            B = scaling_factor*Ly_min
            t = (A-1)/(A-B)
            y = vector(t*y + (1-t)*x)
            Gy = self.pt_to_rnmatrix(y)
            Gy_int = self.scale_to_integer(Gy)
            Ly_min, v = self.single_min_vec(Gy_int)
            
        return y
    
    def get_y1s(self, x, mvx_orbit_reps):
        cone = self.tangent_cone(x, mvx_orbit_reps)
        # the columns of ray_matrix are the ray vectors
        ray_matrix = Matrix(QQ,[vector(r) for r in cone.rays()]).T
        b_values = self.boundary_intersection(x, ray_matrix)
        b_values = [self.ratl_approx(b) for b in b_values]
        rt1s = (ray_matrix*diagonal_matrix(b_values)).T
        y1s = Matrix(QQ,[rt1 + x for rt1 in rt1s])
        return y1s

    def boundary_intersection(self, x, ray_matrix):
        # for each ray r of the tangent cone, get the values b
        # such that x + b*r on the boundary of X_p
        face_ray_matrix = (self.face_matrix)*ray_matrix
        invert_face_ray_matrix = face_ray_matrix.apply_map(lambda s: 1/s)
        x_tilde = vector([1]+list(x))
        t_matrix = diagonal_matrix(-self.face_matrix_tilde*x_tilde)*invert_face_ray_matrix
        t_matrix_np = t_matrix.numpy()
        t_matrix_np[t_matrix_np<0] = Infinity
        b_values = vector(np.min(t_matrix_np, axis=0))
        return b_values
    
    def ratl_approx(self, b):
        # approximates b by a rational number smaller than it,
        # rounds down to the nearest self.dec_approx
        ratlb = QQ(floor(self.dec_approx*b)/self.dec_approx)
        return ratlb
    
    def tangent_cone(self, x, mvx_orbit_reps):
        # gets the tangent cone to a vertex of Tn as a polyhedron
        adjacent_elements = [self.intvec_to_Rp(v) for v in mvx_orbit_reps]
        inequalities = self.get_inequalities(adjacent_elements)
        cone = Polyhedron(ieqs = inequalities.values(), base_ring = QQ, backend = "normaliz")
        return cone
    
    @staticmethod
    def scale_to_integer(vec):
        # scale a rational vector or matrix to be integral
        common_denom = lcm([x.denominator() for x in vec.list()])
        return common_denom*vec

    def pt_to_rnmatrix(self, x):
        # given a point x in X_p, return the corresponding cyclic R_p-matrix
        x = list(x)
        if self.p%2 == 1:
            a_mid = -1/2 - sum(x)
            a = list([1] + x + [a_mid, a_mid] + list(reversed(x)))
        if self.p%2 == 0:
            a_mid = -1/2 - sum(x)
            a = list([1] + x + [a_mid] + list(reversed(x)))
        C = matrix.circulant(a)
        G = C[:self.p-1,:self.p-1]
        return G

    def single_min_vec(self, Gy_int):
        # returns a single minimal vector of Qy and its length
        Qy = QuadraticForm(Gy_int)
        _, Ly_min, v = Qy.__pari__().qfminim(m=1)
        v = vector(QQ,list(*v))
        Ly_min = Integer(Ly_min)
        return Ly_min, v
    
    def min_vecs_cn_orbit_reps(self, Gx_int):
        # calculates a set of orbit representatives for the actions of C_n
        # on the minimal vectors of L^x
        Qx = QuadraticForm(Gx_int)
        _, Lx_min, mvx = Qx.__pari__().qfminim()
        Lx_min = Integer(Lx_min)
        orbits = va.Cn_matrix.__pari__().qforbits(mvx)
        orbit_reps = [vector(QQ,orbit[0]) for orbit in orbits]
        return Lx_min, orbit_reps
    
    def intvec_to_Rp(self, v):
        # converts integer vector to element of Rp = OK
        alpha = sum([vi*self.zeta**i for i, vi in enumerate(v)])
        return alpha

    def get_inequalities(self, elements):
        """
        Given a list of candidate elements of Rp, returns a list of resulting inequalities
        to use as input for Sage's Polytope constructor
        """
        inequalities = {}
        for alpha in elements:
            if alpha in self.global_inequalities:
                inequalities[alpha] = self.global_inequalities[alpha]
            else:
                cx_norm_alpha = alpha*(self.cc(alpha))
                e = [self.sigma[i](cx_norm_alpha) for i in range(self.totally_real_degree)]
                A_b1 = vector(e)*self.Minv
                A_b1[0] = A_b1[0] - 1
                vec_ineq_alpha = vector(QQ,A_b1)
                inequalities[alpha] = vec_ineq_alpha
                torsion_orbit_alpha = [(-1)^ii*(self.zeta)^jj*alpha for ii in range(2) for jj in range(self.p)]
                for beta in torsion_orbit_alpha:
                    self.global_inequalities[beta] = vec_ineq_alpha
        return inequalities
    
    def process_vertices(self, vertices):
        vertices_df = pd.DataFrame(data=list(vertices))
        vertices_df["coords"] = vertices_df.values.tolist()
        vertices_df = vertices_df["coords"].to_frame()
        vertices_df["coords"] = vertices_df["coords"].apply(vector)
        vertices_df = vertices_df["coords"].to_frame()
        vertices_df["center_density"] = vertices_df["coords"].apply(self.spd)
        vertices_df = vertices_df.groupby("center_density")["coords"].apply(list)
        vertices_df = vertices_df.to_frame().reset_index().sort_values("center_density", ascending=False)
        vertices_df["equivalence_classes"] = vertices_df["coords"].apply(self.sort_into_equivalence_classes)
        vertices_df = vertices_df.explode("equivalence_classes").reset_index(drop=True)
        vertices_df["num_vertices"] = vertices_df["equivalence_classes"].apply(len)
        vertices_df["equivalence_class_rep"] = vertices_df["equivalence_classes"].apply(lambda lst: lst[0])
        vertices_df = vertices_df[["equivalence_class_rep", "center_density", "num_vertices"]]
        vertices_df["kissing_number"] = vertices_df["equivalence_class_rep"].apply(self.kissing_number)
        vertices_df["minimal_vectors"] = vertices_df["equivalence_class_rep"].apply(self.alg_min_vecs)
        vertices_df["algebraic_norms"] = vertices_df["minimal_vectors"].apply(lambda lst: set([alpha.norm() for alpha in lst]))
        return vertices_df
        
    def spd(self, x):
        # computes the sphere-packing (center) density of the lattice Lx
        # where x is a vertex of Tp
        Gx = self.pt_to_rnmatrix(x)
        return RR((1/2)^(self.p-1)/(Gx.det()^(1/2)))
    
    def kissing_number(self, x):
        # computes the kissing number of the lattice Lx
        Qx = QuadraticForm(self.scale_to_integer(self.pt_to_rnmatrix(x)))
        kiss, _, _ = Qx.__pari__().qfminim()
        return Integer(kiss)
    
    def sort_into_equivalence_classes(self, vertex_list):  
        equiv_classes = []
        vertex_list_copy = vertex_list.copy()
        while len(vertex_list_copy) > 0:
            x = vertex_list_copy[0]
            equiv_class = []
            Qx = QuadraticForm(self.scale_to_integer(self.pt_to_rnmatrix(x)))
            Qx_isom = Qx.__pari__().qfisominit()
            for y in vertex_list_copy:
                Qy = QuadraticForm(self.scale_to_integer(self.pt_to_rnmatrix(y)))
                is_equiv = Qx_isom.qfisom(Qy)
                if is_equiv:
                    equiv_class.append(y)
            equiv_classes.append(equiv_class)
            vertex_list_copy = [z for z in vertex_list_copy if z not in equiv_class]
        return equiv_classes
    
    def alg_min_vecs(self, x):
        Gx = self.pt_to_rnmatrix(x)
        Gx_int = self.scale_to_integer(Gx)
        _, mvx_orbit_reps = self.min_vecs_cn_orbit_reps(Gx_int)
        adjacent_elements = [self.intvec_to_Rp(v) for v in mvx_orbit_reps]
        return adjacent_elements

## Run Voronoi's Algorithm for p = 5, p = 7

In [6]:
p = 5
va = VoronoisAlgorithm(p = p, dec_approx=1e4)

In [7]:
%%time
x0 = vector([-1/2] + (va.p0-1)*[0])
vertices = va.va_main(x0=x0)

CPU times: user 10.6 ms, sys: 171 µs, total: 10.8 ms
Wall time: 10.7 ms


In [8]:
vertices

{(-1/2,), (0,)}

In [9]:
p = 7
va = VoronoisAlgorithm(p = p, dec_approx=1e4)

In [10]:
%%time
x0 = vector([-1/2] + (va.p0-1)*[0])
vertices = va.va_main(x0=x0)

CPU times: user 51.6 ms, sys: 2.36 ms, total: 53.9 ms
Wall time: 52.7 ms


In [11]:
vertices

{(-1/2, 0), (-1/2, 1/4), (-1/4, -1/2), (0, -1/2), (0, 0), (1/4, -1/4)}

## Further Results for p = 11

In [14]:
p = 11

### Vertices of T_p sorted into equivalence classes, as a pandas DataFrame

In [16]:
with open(f"p_{p}/equiv_classes_{p}.pkl", "rb") as f:
    ec_df = pkl.load(f)

In [17]:
ec_df

,equivalence_class_rep,center_density,num_vertices,kissing_number,minimal_vectors,algebraic_norms
0,"[-1/2, 0, 0, 1/4]",0.0274101222343421,10,220,"[zeta^9, zeta^9 + zeta^8, zeta^9 + zeta^8 + ze...",{1}
1,"[1/2, 1/10, -1/5, -2/5]",0.0221220640450205,5,132,"[zeta^9, -zeta^9 + zeta^8, zeta^9 + zeta^4, ze...","{1, 11}"
2,"[1/4, -1/8, -1/8, 0]",0.0208387970334523,30,132,"[zeta^9, zeta^9 + zeta^4, zeta^9 - zeta^8 + ze...",{1}
3,"[1/6, -1/3, 0, 1/6]",0.0189223287015487,5,110,"[zeta^9, zeta^9 + zeta^4, zeta^9 + zeta^7 + ze...",{1}
4,"[0, 0, -1/2, 0]",0.00942222951805511,5,110,"[zeta^9, zeta^9 + zeta^6, zeta^9 + zeta^6 + ze...",{1}


### T_p as a SageMath polytope object

In [20]:
with open(f"p_{p}/T_{p}.pkl", "rb") as f:
    Tp_polytope = pkl.load(f)

In [21]:
Tp_polytope

A 4-dimensional polyhedron in QQ^4 defined as the convex hull of 55 vertices (use the .plot() method to plot)

### Vertices of T_p

In [22]:
with open(f"p_{p}/vertices_{p}.pkl", "rb") as f:
    vertices = pkl.load(f)

In [23]:
vertices

{(-1/2, 0, 0, 0),
 (-1/2, 0, 0, 1/4),
 (-1/2, 0, 1/4, -1/2),
 (-1/2, 0, 3/8, -1/2),
 (-1/2, 1/8, -1/4, 3/8),
 (-1/2, 1/6, 1/6, -1/3),
 (-1/2, 1/4, -1/2, 1/4),
 (-1/2, 1/4, -3/8, 0),
 (-1/2, 1/4, 0, -1/8),
 (-1/2, 3/8, -1/2, 1/8),
 (-1/2, 3/8, -3/8, 1/4),
 (-1/2, 3/8, -1/8, -1/8),
 (-1/2, 1/2, -2/5, 1/10),
 (-2/5, -1/5, 1/2, -1/2),
 (-3/8, -1/4, 3/8, -1/2),
 (-3/8, 1/8, 1/4, -1/2),
 (-1/3, 1/6, -1/2, 0),
 (-1/4, -1/2, 1/4, 0),
 (-1/4, -1/2, 1/4, 3/8),
 (-1/4, -1/2, 3/8, 1/8),
 (-1/4, -1/4, 1/8, -1/2),
 (-1/5, -1/2, 1/10, 1/2),
 (-1/8, -1/2, -1/8, 1/4),
 (-1/8, -1/2, -1/8, 3/8),
 (-1/8, -1/8, -1/2, -1/8),
 (-1/8, -1/8, 3/8, -1/2),
 (-1/8, 0, -1/2, -1/8),
 (0, -1/2, -1/3, 1/6),
 (0, -1/2, 0, 0),
 (0, -1/2, 1/8, 3/8),
 (0, -1/2, 1/4, 1/4),
 (0, -3/8, -1/2, 1/8),
 (0, -1/4, 0, -1/2),
 (0, -1/8, 1/4, -1/2),
 (0, 0, -1/2, 0),
 (0, 0, 0, -1/2),
 (0, 0, 0, 0),
 (0, 1/4, -1/4, 0),
 (1/10, -2/5, -1/2, -1/5),
 (1/8, -1/2, -1/2, 0),
 (1/8, -1/2, 0, 1/4),
 (1/8, 3/8, -1/4, -1/4),
 (1/6, -1/3, 0, 1/6